In [1]:
import re
import string
import nltk
import numpy as np
import copy

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn import svm

In [3]:
wordlist_file = open('parsed.txt', 'r', encoding = 'utf-8').read().split('\n')

In [4]:
wordlist = {}

In [5]:
#здесь собираются все возможные разборы слов
for line in wordlist_file:
    word = re.findall('</ana>([^<]*?)</w>', line)
    if word:
        analyses = re.findall('<ana (.*?)>', line)
        if analyses:
            wordlist[word[0]] = []
            for i in range(len(analyses)):
                gr = re.findall('gr="(.*?)"', analyses[i])[0].split(',')
                lex = re.findall('lex="(.*?)"', analyses[i])[0]
                wordlist[word[0]].append((word[0], lex, tuple(gr)))

In [6]:
wordlist['мужикеным']

[('мужикеным', 'мужик', ('N', 'rus', 'anim', 'hum', 'sg', 'ins', '1sg'))]

In [97]:
#все признаки существительных
grams_N = []
for k in wordlist:
    for el in wordlist[k]:
        if el[2][0] == 'N':
            for gr in el[2][1:]:
                if gr not in grams_N and 'sg' not in gr and 'pl' not in gr and 'rus' not in gr and 'attr' not in gr and '_' not in gr:
                    grams_N.append(gr)

In [99]:
grams_N = ['anim', 'nom', 'ins', 'loc', 'ill', 'el', 'hum', 'PN',
           'topn', 'prol', 'vn', 'body', 'adv', 'egr', 'dat', 'acc',
           'gen', 'app', 'transport', 'abl', 'car', 'term']

In [8]:
#все признаки глаголов
grams_V = []
for k in wordlist:
    for el in wordlist[k]:
        if el[2][0] == 'V':
            for gr in el[2][1:]:
                if gr not in grams_V:
                    grams_V.append(gr)

In [102]:
grams_V = ['pst', 'prs', '3', 'tr', 'cvb', 'gen', 'intr', 'with_ill',
           'sg', 'fut', 'evid', '2', 'ptcp', 'nom', 'imp', 'pl',
           'neg', '1', 'with_el', 'vn', 'with_instr', 'inf',
           'with_dat', 'act', 'acc', '3sg', 'with_abl', 'ins',
           'with_inf', 'pass', 'deb', 'loc', 'res', 'caus', 'ill',
           '1sg', 'adv', 'dur', 'subj', 'el', 'iter', 'impers', 'mult',
           '2pl', '3pl', 'dat', 'comp', 'hort', 'abl', '1pl', 'lim',
           'egr', '2sg', 'prol', 'detr', 'term', 'car', 'refl']

In [49]:
post = {}
for line in wordlist_file:
    word = re.findall('</ana>([^<]*?)</w>', line)
    if word:
        analyses = re.findall('<ana (.*?)>', line)
        if analyses:
            for el in analyses:
                gr = re.findall('gr="(.*?)"', el)[0].split(',')
                if 'POST' in gr:
                    trans = re.findall('trans_ru="(.*?)"', el)[0]
                    lex = re.findall('lex="(.*?)"', el)[0]
                    post[lex] = trans

In [127]:
def parse_sentence(s, wordlist, models, grams_N, grams_V):
    words = [el.lower() for el in nltk.word_tokenize(s)]
   # print(words)
    analyses = []
    for word in words:
        try:
            analyses.append(wordlist[word])
        except KeyError:
            if word.isdigit():
                analyses.append([(word, word, 'NUM')])
            else:
                analyses.append([])
    predicates = find_predicate(analyses, models) #находим предикат
 #   print(1)
    analyses = drop_attr(analyses) #соединяем прил и числ с существительными
    analyses = drop_post(analyses) #cоединяем послелоги с сущ
  #  print(2)

   # print(analyses)
    new_ana = []
    for ana in analyses:
        new_ana.append(list(set(ana)))
 #   print(new_ana)


    
    #return analyses, predicates
    args = find_args(new_ana, models, predicates) #находим все аргументы в соответствии с правилами
    #print(args)
    
    X = collect_X(analyses, predicates, grams_N, grams_V)
    return X, args

In [109]:
def drop_post(analyses):
    analyses2 = []
    for i in range(len(analyses)):
        analyses2.append([])
        for el in analyses[i]:
            if 'POST' in el[2] or 'rel_n' in el[2]:
                try:
                    for k in analyses2[-2]:
                        if ('N' in k[2] or 'PRO' in k[2] or 'V' in k[2]):
                            analyses2[-1].append((k[0] + ' ' + el[0], el[1], el[2]))
                            analyses2[-2] = []
                except IndexError:
                    pass
            else:
                analyses2[-1].append(el)
    ana3 = [x for x in analyses2 if x != []]
    return ana3

In [170]:
def drop_attr(analyses):
    analyses2 = []
    #print(analyses)
    for i in range(len(analyses)):
        analyses2.append([])
      #  print(analyses[i])
        for el in analyses[i]:
       #     print(el)
            if 'N' in el[2]:
                try:
                    for k in analyses2[-2]:
        #                print(k)
                        if 'ADJ' in k[2] or 'NUM' in k[2] or 'ADJPRO' in k[2] or ('N' in k[2] and ('nom' in k[2])) or (('N' in k[2] or 'PRO' in k[2]) and ('gen' in k[2])) or ('ptcp' in k[2]):
                                
                            analyses2[-2] = []
                            analyses2[-1].append((k[0] + ' ' + el[0], el[1], el[2]))
                            #analyses2.remove(analyses2[-1])
                    if analyses2[-1] == []:
                        test_copy = copy.copy(analyses[i])
                        analyses2[-1] = test_copy
                except IndexError:
                    analyses2[-1].append(el)
            else:
                analyses2[-1].append(el)
    ana3 = [x for x in analyses2 if x != []]
    return ana3

In [ ]:
здесь прописать для причастия и дериваций

In [12]:
def find_predicate(analyses, models):
    predicates = []
    for i in range(len(analyses)):
        for ana in analyses[i]:
            lemma = ana[1]
            if lemma in models:
                predicates.append(ana + (i,))
    return predicates
                

In [153]:
def find_args(analyses, models, predicates):
    args = []
#    print(analyses)
    for pred in predicates:
      #  print(pred[0])
        model = models[pred[1]]
        for el in analyses:
            for ana in el:
                i = 0
                if ana[0] != pred[0]:
                    for key in model:
                        if key in ana[2] or key in ana[1]:
                            if ('1' in pred[2] or '2' in pred[2]) and 'nom' in ana[2]:
                                if (ana[1] == 'мон' and '1' in pred[2] and 'sg' in pred[2]) or (ana[1] == 'ми' and '1' in pred[2] and 'pl' in pred[2]) or (ana[1] == 'тон' and '2' in pred[2] and 'sg' in pred[2]) or (ana[1] == 'тӥ' and '2' in pred[2] and 'pl' in pred[2]):
                                    args.append([ana, pred, model[key]])
                            else:
                                args.append([ana, pred, model[key]])
                      #      print(model[key] + ' - ' + str(ana))
                            i = 1
                    if i == 0:
                        for p in post:
                            if ana[0].endswith(p) and ('rel_n' in ana[2] or 'POST' in ana[2]):
                                args.append([ana, pred, post[p]])
 #   print(analyses, predicates, args)
                       #        print(post[p] + ' - ' + str(ana))
    return args

In [14]:
def collect_X(analyses, predicates, grams_N, grams_V):
    X = []
    for pred in predicates:
        for i in range(len(analyses)):
            for ana in analyses[i]:
                if ana[0] != pred[0]:
                    X.append([])
                    X[-1].append(ana)
              #      print(X[-1])
                    X[-1].append(pred)
                #    print(X[-1])
                    X[-1].append(i - pred[3])
                 #   print(X[-1])
                    #X[-1].append(ana[2])
                  #  print(X[-1])
                   # for gr in grams_V:
                    #    if gr in ana[2]:
                     #       X[-1].append(1)
                      #  else:
                       #     X[-1].append(0)
           #         for gr in grams_V:
            #            if gr in pred[2]:
             #               X[-1].append(1)
              #          else:
               #             X[-1].append(0)
                   # X[-1].append(pred[2])
                    #print(X[-1])
    return X

In [187]:
a, b = parse_sentence('Ува ёросын нуналлы быдэ 4-5 бригадаос шуныт сётӥсь гумыосты воштон бордын ужало.', wordlist, models, grams_N, grams_V)

In [188]:
a, b

([[('ува ёросын', 'ёрос', ('N', 'sg', 'loc')),
   ('сётӥсь', 'сётыны', ('V', 'I', 'tr', 'with_dat', 'ptcp', 'act'), 7),
   -7],
  [('нуналлы быдэ', 'быдэ', ('POST',)),
   ('сётӥсь', 'сётыны', ('V', 'I', 'tr', 'with_dat', 'ptcp', 'act'), 7),
   -6],
  [('бригадаос', 'бригада', ('N', 'rus', 'pl', 'nom')),
   ('сётӥсь', 'сётыны', ('V', 'I', 'tr', 'with_dat', 'ptcp', 'act'), 7),
   -5],
  [('шуныт', 'шуныт', ('ADJ',)),
   ('сётӥсь', 'сётыны', ('V', 'I', 'tr', 'with_dat', 'ptcp', 'act'), 7),
   -4],
  [('шуныт', 'шуныт', ('ADV',)),
   ('сётӥсь', 'сётыны', ('V', 'I', 'tr', 'with_dat', 'ptcp', 'act'), 7),
   -4],
  [('сётӥсь гумыосты', 'гумы', ('N', 'pl', 'acc')),
   ('сётӥсь', 'сётыны', ('V', 'I', 'tr', 'with_dat', 'ptcp', 'act'), 7),
   -3],
  [('воштон бордын', 'борд', ('N', 'rel_n', 'sg', 'loc')),
   ('сётӥсь', 'сётыны', ('V', 'I', 'tr', 'with_dat', 'ptcp', 'act'), 7),
   -2],
  [('ужало', 'ужаны', ('V', 'II', 'intr', 'with_instr', '1', 'sg', 'fut')),
   ('сётӥсь', 'сётыны', ('V', 'I', 't

In [18]:
verbs = open('verbs.csv', 'r', encoding = 'utf-8').read().split('\n') #модели управления 20 частотных глаголов

In [19]:
verbs[0]

'ужаны\tV,II,intr,with_ins\tработать\targ0\tagent\tnom'

In [20]:
models = {}

In [21]:
for line in verbs:
    line = line.split('\t')
    if line[0]:
        if line[0] not in models:
            models[line[0]] = {}
        models[line[0]][line[5]] = line[4]

In [22]:
models

{'басьтыны': {'abl': 'source',
  'acc': 'patient',
  'el': 'source',
  'ill': 'goal',
  'ins': 'price/manner',
  'loc': 'source',
  'nom': 'agent/patient'},
 'валаны': {'acc': 'content/patient',
  'clause': 'content/patient',
  'nom': 'agent'},
 'вераны': {'acc': 'content',
  'clause': 'content',
  'dat': 'recipient',
  'ins': 'manner',
  'nom': 'agent',
  'сярысь': 'theme'},
 'ветлыны': {'el': 'source',
  'ill': 'goal',
  'ins': 'manner/partner',
  'loc': 'location',
  'nom': 'agent',
  'prol': 'location'},
 'карыны': {'acc': 'patient', 'nom': 'agent'},
 'кутыны': {'acc': 'patient',
  'ill': 'goal',
  'ins': 'instrument',
  'nom': 'agent'},
 'лэсьтыны': {'acc': 'patient', 'ins': 'instrument', 'nom': 'agent'},
 'поттыны': {'acc': 'patient',
  'el': 'source',
  'ill': 'goal',
  'ins': 'manner',
  'nom': 'agent'},
 'потыны': {'el': 'source', 'ill': 'goal', 'nom': 'agent'},
 'пуктыны': {'acc': 'patient', 'ill': 'goal', 'ins': 'manner', 'nom': 'agent'},
 'пырыны': {'el': 'source', 'ill': '

In [52]:
#послелоги
post = {}
with open('post.txt', 'r', encoding = 'utf-8') as postfile:
    f = postfile.read().split('\n')
    for line in f:
        line = line.split('\t')
        post[line[0]] = line[2]

In [114]:
examples = open('examples.txt', 'r', encoding = 'utf-8').read().split('\n')

In [115]:
len(examples)

847

In [116]:
examples[1]

'Ува ёросын нуналлы быдэ 4-5 бригадаос шуныт сётӥсь гумыосты воштон бордын ужало.'

In [117]:
short_ex = []
for ex in examples:
    if len(ex) <= 80:
        short_ex.append(ex)
        

In [118]:
len(short_ex)

447

In [171]:
X, args = [], []

In [172]:
y = []

In [173]:
for ex in short_ex:
    ex_X, ex_args = parse_sentence(ex, wordlist, models, grams_N, grams_V)
    X += ex_X
    args += ex_args
    
    ex_y = []
    for i in range(len(ex_X)):
        for j in ex_args:
            #print(ex_X[i][:2], j[:2])
            if ex_X[i][:2] == j[:2]:
                ex_y.append(j[2])
                break
        #print(len(ex_y), i)
        if len(ex_y) < i+1:
            ex_y.append(0)
        #print(i[0], j[0], i[1], j[1])
         #   else:
          #      y.append(0)
    y += ex_y

    

In [174]:
len(X)

9480

In [175]:
X[0]

[('татын', 'татын', ('ADVPRO', 'loc')),
 ('ужаськомы',
  'ужаны',
  ('V', 'II', 'intr', 'with_instr', '1', 'pl', 'prs'),
  7),
 -7]

In [176]:
args[0]

[('татын', 'татын', ('ADVPRO', 'loc')),
 ('ужаськомы',
  'ужаны',
  ('V', 'II', 'intr', 'with_instr', '1', 'pl', 'prs'),
  7),
 'location']

In [177]:
new_X = []
for el in X:
    a = [el[0][0], el[1][0], el[2]]
    for gr in grams_V:
        if gr in el[0][2]:
            a.append(1)
        else:
            a.append(0)
    for gr in grams_V:
        if gr in el[1][2]:
            a.append(1)
        else:
            a.append(0)
    new_X.append(a)

In [178]:
len(new_X[0])

119

In [179]:
y[0]

'location'

In [180]:
set(y)

{0,
 'recipient',
 'source',
 'manner/partner',
 'partner',
 'manner',
 'instrument',
 'price/manner',
 'goal',
 'time',
 'content/patient',
 'patient',
 'theme',
 'content',
 'location',
 'agent',
 'agent/patient',
 'status/instrument/partner'}

In [181]:
s = ''
for i in range(len(new_X)):
    x = '\t'.join([str(el) for el in new_X[i]])
    s = s + x + '\t' + str(y[i]) + '\n'
    

In [182]:
len(y)

9480

In [183]:
f = open('pairs-2.csv', 'w', encoding = 'utf-8')
f.write(s) 
f.close() 

In [184]:
f = open('pairs-2.csv', 'r', encoding = 'utf-8')
lines = list(set(f.read().split('\n')))
f.close() 

f = open('pairs-2.csv', 'w', encoding = 'utf-8')
f.write('\n'.join(lines)) 
f.close() 

In [189]:
new_X = np.array(new_X)

In [190]:
new_X

array([['татын', 'ужаськомы', '-7', ..., '0', '0', '0'],
       ['ми', 'ужаськомы', '-6', ..., '0', '0', '0'],
       ['мужикеным', 'ужаськомы', '-5', ..., '0', '0', '0'],
       ..., 
       ['азинтыны', 'утьыны', '2', ..., '0', '0', '0'],
       ['калыклэсь', 'утьыны', '3', ..., '0', '0', '0'],
       ['лулчеберетсэ', 'утьыны', '4', ..., '0', '0', '0']],
      dtype='<U40')

In [7]:
f = open('pairs-3.csv', 'r', encoding = 'utf-8')
lines = f.read().split('\n')
new_y = [i.split('\t')[-1] for i in lines] 
new_X = [i.split('\t')[2:-2] for i in lines] 
f.close() 

In [51]:
text = '\n'.join(short_ex)

In [52]:
f = open('short_ex.txt', 'w', encoding = 'utf-8')
f.write(text) 
f.close() 

In [8]:
X_train, X_test, y_train, y_test = train_test_split(new_X, new_y, test_size=0.3, random_state=42)

In [200]:
X_train = [x[2:] for x in X_train]
X_test = [x[2:] for x in X_test]

In [9]:
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [10]:
y_pred = clf.predict(X_test)

In [11]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

   
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))    

0.277914723008
0.297722488587
0.26781252182


C:\Users\zhenya\A3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\zhenya\A3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
